In [1]:
# Load Libraries
import polars as pl
import polars.selectors as cs
import sys 

# Display system and polars versions
print(f'My system version is {sys.version};\npolars version is {pl.__version__}')

My system version is 3.12.4 (main, Jul  1 2024, 00:48:18) [Clang 15.0.0 (clang-1500.3.9.4)];
polars version is 1.2.1
My system version is 3.12.4 (main, Jul  1 2024, 00:48:18) [Clang 15.0.0 (clang-1500.3.9.4)];
polars version is 1.2.1
